In [14]:
import pandas as pd
import networkx as nx
import numpy as np
import glob
import natsort
import math
import copy
import os

### Import files

In [15]:
# Set directories
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
input_path = os.path.join(parent_dir, "_ModelsAndNetworks")

In [16]:
Edge_filenames = natsort.natsorted(glob.glob(input_path + "/*Edgelist.txt"))
Species = ['_'.join(j.split('.')[0].split('_')[:-1]) for j in [i.split('\\')[-1] for i in Edge_filenames]]


Met_adjs = []
for i in range(len(Species)):
    M_edge = pd.read_table(Edge_filenames[i], header = None)
    M_edge.columns = ['source', 'target']
    M_network = nx.from_pandas_edgelist(M_edge, create_using=nx.DiGraph)
    M_ad = nx.to_pandas_adjacency(M_network)
    Met_adjs.append(M_ad)

Species

['Actinomyces_odontolyticus_ATCC_17982',
 'Alistipes_putredinis_DSM_17216',
 'Anaerococcus_hydrogenalis_DSM_7454',
 'Anaerofustis_stercorihominis_DSM_17244',
 'Anaerostipes_caccae_DSM_14662',
 'Anaerotruncus_colihominis_DSM_17241',
 'Bacteroides_caccae_ATCC_43185',
 'Bacteroides_cellulosilyticus_DSM_14838',
 'Bacteroides_coprophilus_DSM_18228',
 'Bacteroides_dorei_DSM_17855']

### Filter the networks

In [17]:
# Find and remove glycans
glc_Met_adjs = []
glc_Met_IDs = []
for adj in Met_adjs:
    
    ID = list(adj.columns)
    G = nx.from_pandas_adjacency(adj, create_using= nx.DiGraph)

    glcNodes = [i for i in ID if 'MGlcn' in i]

    glc_N = copy.deepcopy(G)
    glc_N.remove_nodes_from(glcNodes)

    glc_adj = nx.to_pandas_adjacency(glc_N)
    glc_ID = list(glc_adj.columns)

    glc_Met_adjs.append(glc_adj)
    glc_Met_IDs.append(glc_ID)
    
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Find and remove currency metabolites
perc = 0.03

curr_Met_adjs = []
curr_Met_IDs = []
for adj0 in glc_Met_adjs:
    ID = list(adj0.columns)
    top_n_nodes = math.ceil(perc*len(ID))

    glc_G = nx.from_pandas_adjacency(adj0, create_using= nx.DiGraph)
    SortedNodes_Degree = sorted(dict(glc_G.degree(glc_G.nodes())).items(), key=lambda x:x[1], reverse=True)
    SortedNodes = [i[0] for i in SortedNodes_Degree]

    #remove
    curr_N = copy.deepcopy(glc_G)
    curr_N.remove_nodes_from(SortedNodes[:top_n_nodes])

    curr_adj = nx.to_pandas_adjacency(curr_N)
    curr_ID = list(curr_adj.columns)

    curr_Met_adjs.append(curr_adj)
    curr_Met_IDs.append(curr_ID)

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Find and remove small weekly connected components
threshold = 10

wcc_Met_adjs = []
wcc_Met_IDs = []
for adj1 in curr_Met_adjs:
    curr_G = nx.from_pandas_adjacency(adj1, create_using= nx.DiGraph)

    sorted_CC_list = [sorted(list(c)) for c in sorted(nx.weakly_connected_components(curr_G), key=len, reverse=True)]
    removenodes = [i for i in sorted_CC_list if len(i) <= threshold]
    removenodes = [val for sublist in removenodes for val in sublist]

    wcc_N = copy.deepcopy(curr_G)
    wcc_N.remove_nodes_from(removenodes)

    wcc_adj = nx.to_pandas_adjacency(wcc_N)
    wcc_ID = list(wcc_adj.columns)

    wcc_Met_adjs.append(wcc_adj)
    wcc_Met_IDs.append(wcc_ID)

### Export filtered adjacency networks

In [18]:
os.makedirs(f'{current_dir}/0_FilteredAdjacency', exist_ok=True)
for m, fil_adj in enumerate(wcc_Met_adjs):
    fil_adj.to_csv(f'{current_dir}/0_FilteredAdjacency/{Species[m]}_FilteredNetwork.csv')

### Find the seed set of each network

In [19]:
SeedSet_confidence_list = []
SeedSet_list = []

for adj_,ID in zip(wcc_Met_adjs, wcc_Met_IDs):

    G_raph = nx.from_pandas_adjacency(adj_, create_using= nx.DiGraph)

    sorted_SCCs_list = [sorted(list(c)) for c in sorted(nx.kosaraju_strongly_connected_components(G_raph), key=len, reverse=True)]

    ## IF WANT TO FILTER THE LARGE SCCs FROM CONSIDERATION AS SEED SETS
    #sorted_SCCs_list = [i for i in sorted_SCCs_list if len(i) <= 5]

    SizeOfSCC = [len(i) for i in sorted_SCCs_list]
    edgelist_ = G_raph.edges()

    source_seed = []
    source_seed_dict = {}
    for nodelist__ in sorted_SCCs_list:
        for edges in edgelist_:
            count = 0
            if (edges[0] not in nodelist__ and edges[1] in nodelist__):
                count += 1
                break
        if count == 0:
            source_seed.append(nodelist__)

    n_source_seed = []
    for sSeed in source_seed:
        temp = 0
        for edge__s in edgelist_:
            for i in sSeed:
                if ((i, edge__s[1]) in edgelist_) & (edge__s[1] not in sSeed):
                    temp += 1
        if temp > 0:
            n_source_seed.append(sSeed)

    SeedSet_list.append(n_source_seed)
    
    seed_confidence = [[(i, 1/len(item)) for i in item] for item in n_source_seed]
    Flat_seed_confidence = [val for sublist in seed_confidence for val in sublist]            
    SeedSet_confidence_list.append(Flat_seed_confidence)

Species_compound_dict = dict(zip(Species, SeedSet_confidence_list))
Species_set_dict = dict(zip(Species, SeedSet_list))

### Export seed layers

In [20]:
os.makedirs(f'{current_dir}/1_SpeciesMetaLayers', exist_ok=True)
Seed_layer = pd.DataFrame([[i[0] for i in v] for k,v in Species_compound_dict.items()], index=Species).T
Seed_layer.to_csv(f'{current_dir}/1_SpeciesMetaLayers/B-I_S.csv')

### Calculate and export competitive indices

In [21]:
os.makedirs(f'{current_dir}/2_Indices', exist_ok=True)

In [22]:
CompIndexMatrixW = pd.DataFrame(np.zeros((len(Species), len(Species)), dtype=float), columns= list(Species_compound_dict.keys()))
CompIndexMatrixW.index = CompIndexMatrixW.columns
for speci1 in Species_compound_dict.keys():
    for speci2 in Species_compound_dict.keys():

        comp1 = [i[0] for i in Species_compound_dict[speci1]]
        comp2 = [i[0] for i in Species_compound_dict[speci2]]
        intersect = list(set(comp1) & set(comp2))

        # with weights
        numerW = np.sum([item[1] for item in Species_compound_dict[speci1] if item[0] in intersect])
        denomW = np.sum([item[1] for item in Species_compound_dict[speci1]])
        
        CI_W = numerW/denomW
        CompIndexMatrixW.loc[speci1, speci2] = CI_W

CompIndexMatrixW.to_csv(f'{current_dir}/2_Indices/BorensteinImitate_Weighted_CI_SS_Curr{perc}.csv', header=True, index=True)

### Find and export the product set of each network

In [24]:
products = {}
count_0 = 0
for speci in Species_compound_dict.keys():

    SS = [i[0] for i in Species_compound_dict[speci]]
    product_nodes = list(set(wcc_Met_IDs[count_0]) - set(SS))

    products[speci] = [i for i in product_nodes] # if '_c' in i]
    count_0 += 1

Product_layer = pd.DataFrame([v for k,v in products.items()], index = Species).T
Product_layer.to_csv(f'{current_dir}/1_SpeciesMetaLayers/B-I_P.csv')

### Calculate and export synergistic indices

In [25]:
SynIndexMatrixW = pd.DataFrame(np.zeros((len(Species), len(Species)), dtype=float), columns= list(Species_compound_dict.keys()))
SynIndexMatrixW.index = SynIndexMatrixW.columns
for speci1 in Species_compound_dict.keys():
    for speci2 in Species_compound_dict.keys():
        if speci1 != speci2:

            comp1 = [i[0] for i in Species_compound_dict[speci1]]
            intersect_ = list(set(comp1) & set(products[speci2]))
            
            # with weights
            numerW = np.sum([item[1] for item in Species_compound_dict[speci1] if item[0] in intersect_])
            denomW = np.sum([item[1] for item in Species_compound_dict[speci1]])

            SI_W = numerW/denomW

            # print(f'{speci1} -> {speci2}: SI = {SI}')
            # rows should be speci1 and columns speci2
            SynIndexMatrixW.loc[speci1, speci2] = SI_W

SynIndexMatrixW.to_csv(f'{current_dir}/2_Indices/BorensteinImitate_Weighted_SI_PS_Curr{perc}.csv', header=True, index=True)